In [1]:
import itertools
from time import sleep
import numpy as np
import os
import pandas as pd
import re
import pathlib
import csv
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.metrics import precision_score
import shutil
import soundfile as sf
import matplotlib.pyplot as plt
import librosa
from pathlib import Path
import keyboard

2022-10-23 23:02:27.274554: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-23 23:02:27.274609: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
DATASET_ROOT = os.path.join(os.path.expanduser("~"),'dataSet/audio/agender_distribution/')
NETWORK_ROOT = os.path.join(os.path.expanduser("~"),'Mestrado-PC/github/Conv1D/CNN/')

In [ ]:
# pause() function definition.
def pause():
	while True:
		if keyboard.read_key() == 'space':
			# If you put 'space' key
			# the program will resume.
			break

In [3]:
def audio_to_fft(audio):
    # Since tf.signal.fft applies FFT on the innermost dimension, we need to squeeze the dimensions and then expand them again after FFT
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64))
    fft = tf.expand_dims(fft, axis=-1)
    # Return the absolute value of the first half of the FFT which represents the positive frequencies
    return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])

In [4]:
def path_to_audio(path):
    # Reads and decodes an audio file
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, 8000)
    return audio

In [5]:
def paths_and_labels_to_dataset(audio_paths, labels):
    # Constructs a dataset of audios and labels
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))

In [6]:
def map_func(npy_path):
    npy_content = np.load(npy_path)
    return npy_content

In [7]:
def paths_and_labels_to_dataset_HTK(paths, labels):
    path_ds = tf.data.Dataset.from_tensor_slices(paths)
    array_ds = path_ds.map(lambda x: tf.numpy_function(map_func, [x], tf.float64),num_parallel_calls=tf.data.experimental.AUTOTUNE)
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((array_ds, label_ds))

In [8]:
def npy_header_offset(npy_path):
    with open(str(npy_path), 'rb') as f:
        if f.read(6) != b'\x93NUMPY':
            raise ValueError('Invalid NPY file.')
        version_major, version_minor = f.read(2)
        if version_major == 1:
            header_len_size =2
        elif version_major == 2:
            header_len_size = 4
        else:
            raise ValueError('Unknown NPY file version {}.{}.'.format(version_major, version_minor))
        header_len = sum(b << (8 * i) for i, b in enumerate(f.read(header_len_size)))
        header = f.read(header_len)
        if not header.endswith(b'\n'):
            raise ValueError('Invalid NPY file.')
        return f.tell()

In [9]:
train_file_list_path = 'file_lists/3-classes/train_database_full.csv'
test_file_list_path = 'file_lists/3-classes/test_database_full.csv'

In [10]:
train_file_list = pd.read_csv(os.path.join(NETWORK_ROOT, train_file_list_path))
train_audio_files = train_file_list['file']
train_classes = train_file_list['class']

In [11]:
audio_paths = list(train_audio_files)
labels = list(train_classes)

In [ ]:
for i in range(len(audio_paths)):
    audio_paths[i] = re.sub('.wav', '-n.mfc.csv', audio_paths[i])

In [ ]:
for i in range(len(audio_paths)):
    audio_paths[i] = re.sub('-n.npy', '-n.wav', audio_paths[i])

In [12]:
for i in range(len(audio_paths)):
    audio_paths[i] = os.path.join(DATASET_ROOT, audio_paths[i])

In [15]:
train_ds = paths_and_labels_to_dataset(audio_paths, labels)

In [14]:
path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)

In [16]:
train_ds = train_ds.shuffle(buffer_size=128 * 8, seed=43).batch(128)

In [17]:
train_ds = train_ds.map(lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [52]:
batch = train_ds.take(255)

In [ ]:
for element in path_ds:
    a = tf.io.read_file(element)
    # b = tf.io.decode_raw(a, out_type=float, little_endian=False)
    print(element)
    print(a)
    '''b = tf.strings.to_number(tf.strings.split(a, sep=" "), tf.float64)
    print(type(b))
    print(b)'''
    input("Press any key to terminate the program")

In [53]:
batch_elem = list(batch.as_numpy_iterator())

In [55]:
len(batch_elem[254][1])

15